In [176]:
import numpy as np
import sympy as sym
import json
import matplotlib.pyplot as plt
from scipy import linalg
from scipy.interpolate import interp1d
from IPython.display import display, IFrame, HTML

In [177]:
def lqr(A, B, Q, R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K

In [178]:
# FIXME (copy/paste definition of A, B, C, D)
A = np.array([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 9.81, 0.0, 0.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, -9.81, 0.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]])

B = np.array([[0.0, 0.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0],
              [0.0, 0.0, 1.0, 0.0],
              [0.0, 1.0, 0.0, 0.0],
              [1.0, 0.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 1.0]])

C = np.array([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.18511135901176, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.18511135901176, 0.0],
              [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]])

D = np.array([[0.0, -4.09255567950588, 0.0, 0.0],
              [4.09255567950588, 0.0, 0.0, 0.0],
              [0.0, 0.0, 0.0, 0.0]])

In [179]:
s_obs_index = [
    2, # o_z
    4, # theta
    5, # phi
    6, # v_x
    7, # v_y
    8, # v_z
]

In [180]:
A_obs = A[s_obs_index, :][:, s_obs_index]
B_obs = B[s_obs_index, :]
C_obs = C[:, s_obs_index]
D_obs = D

In [181]:
# FIXME:
#
# (1) Change the size of each matrix to match the number of
#     states and outputs for your observable subsystem
#
# (2) Change the value of each diagonal entry based on your
#     results from Lab 7.

# Q = np.diag([
#     (1/(3.377)**2),             # n_x
#     (1/(3.645)**2),             # n_y
#     (1./0.002**2),             # r
# ])

Q = np.diag([
    ((1/2)**2),             # n_x
    ((1/2)**2),             # n_y
    (1/0.002)**2,             # r
])


R = np.diag([
    (1/0.039**2), # o_z
    (1/0.033**2), # theta
    (1/0.042**2), # phi
    (1/0.314**2), # v_x
    (1/0.239**2), # v_y
    (1/0.253**2), # v_z
])

# R = np.diag([
#     (1/0.041)**2, # o_z
#     (1/0.075)**2, # theta
#     (1/0.06)**2, # phi
#     (1/0.287)**2, # v_x
#     (1/0.296)**2, # v_y
#     (1/0.236)**2, # v_z
# ])

# R = np.diag([
#     (1/0.041)**2, # o_z
#     (1/0.075)**2, # theta
#     (1/0.06)**2, # phi
#     (1/1.5)**2, # v_x
#     (1/1.5)**2, # v_y
#     (1/0.236)**2, # v_z
# ])

In [182]:
L_weight = lqr(A_obs.T, C_obs.T, linalg.inv(R), linalg.inv(Q)).T
np.round(L_weight, 10)

array([[ 0.00000000e+00,  0.00000000e+00,  2.51644591e+01],
       [ 1.65000000e-02,  0.00000000e+00,  0.00000000e+00],
       [-0.00000000e+00, -2.10000000e-02, -0.00000000e+00],
       [ 2.53377348e-01,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  2.54200687e-01,  0.00000000e+00],
       [-0.00000000e+00,  0.00000000e+00,  1.26500000e+02]])

In [183]:
A_obs = sym.Matrix(A_obs)
B_obs = sym.Matrix(B_obs)
C_obs = sym.Matrix(C_obs)
D_obs = sym.Matrix(D_obs)

In [184]:
L = sym.Matrix(np.round(L_weight, 10))

In [185]:
#Define Constants
g = 9.81
k_flow = 4.09255568
o_z_eq = 0.1 # <-- FIXME: change equilibrium height
dt = 0.01

In [186]:
o_z, theta, phi, v_x, v_y, v_z = sym.symbols('o_z, theta, phi, v_x, v_y, v_z')
w_x, w_y, w_z, a_z, g = sym.symbols('w_x, w_y, w_z, a_z, g')
n_x, n_y, r = sym.symbols('n_x, n_y, r')

s_hat = sym.Matrix([o_z, theta, phi, v_x, v_y, v_z])
i_i = sym.Matrix([w_x, w_y, w_z, a_z - g])
o = sym.Matrix([n_x, n_y, r])

state_error = A_obs @ (s_hat) + B_obs @ (i_i) # Ax + Bu
sensor_error = (C_obs @ (s_hat) + D_obs @ (i_i) - o) # Cx + Du - y
s_hat_dot = state_error - L @ sensor_error # Ax + Bu - L(Cx + Du - y)

In [187]:
sensor_error

Matrix([
[-n_x + 8.18511135901176*v_x - 4.09255567950588*w_y],
[-n_y + 8.18511135901176*v_y + 4.09255567950588*w_x],
[                                       1.0*o_z - r]])

In [188]:
state_error

Matrix([
[        1.0*v_z],
[        1.0*w_y],
[        1.0*w_x],
[     9.81*theta],
[      -9.81*phi],
[1.0*a_z - 1.0*g]])

In [189]:
s_hat_dot

Matrix([
[                             -25.1644590643*o_z + 25.1644590643*r + 1.0*v_z],
[                  0.0165*n_x - 0.135054337423694*v_x + 1.06752716871185*w_y],
[                  -0.021*n_y + 0.171887338539247*v_y + 1.08594366926962*w_x],
[0.2533773485*n_x + 9.81*theta - 2.07392181332363*v_x + 1.03696090666182*w_y],
[  0.2542006866*n_y - 9.81*phi - 2.08066092735825*v_y - 1.04033046367912*w_x],
[                                      1.0*a_z - 1.0*g - 126.5*o_z + 126.5*r]])

In [190]:
for i in range(0, len(sensor_error)) :
    print(sensor_error[i])
    print()

-n_x + 8.18511135901176*v_x - 4.09255567950588*w_y

-n_y + 8.18511135901176*v_y + 4.09255567950588*w_x

1.0*o_z - r



In [191]:
variables = ['o_z', 'theta', 'phi', 'v_x', 'v_y', 'v_z']

In [192]:
for i in range(0, len(s_hat_dot)) :
    o_z_co = float(round(s_hat_dot[i].coeff(o_z), 10))
    theta_co = float(round(s_hat_dot[i].coeff(theta), 11))
    phi_co = float(round(round(s_hat_dot[i].coeff(phi), 11)))
    n_x_co = float(round(s_hat_dot[i].coeff(n_x), 11))
    n_y_co = float(round(s_hat_dot[i].coeff(n_y), 11))
    r_co = float(round(s_hat_dot[i].coeff(r), 7))
    g_co = float(round(s_hat_dot[i].coeff(g)))
    a_z_co = float(round(s_hat_dot[i].coeff(a_z)))
    v_x_co = float(round(s_hat_dot[i].coeff(v_x), 11))
    v_y_co = float(round(s_hat_dot[i].coeff(v_y), 11))
    v_z_co = float(round(s_hat_dot[i].coeff(v_z)))
    w_x_co = float(round(s_hat_dot[i].coeff(w_x), 11))
    w_y_co = float(round(s_hat_dot[i].coeff(w_y), 11))
    w_z_co = float(round(s_hat_dot[i].coeff(w_z)))
    print(variables[i] + ' += ' + 'dt*(' + str(o_z_co) + 'f' + '*o_z + ' +
          str(theta_co) + 'f' + '*theta + ' +
          str(phi_co) + 'f' + '*phi + ' +
          str(n_x_co) + 'f' + '*n_x + ' +
          str(n_y_co) + 'f' + '*n_y + ' +
          str(r_co) + 'f' + '*r + ' +
          str(g_co) + 'f' + '*g + ' +
          str(a_z_co) + 'f' + '*a_z + ' +
          str(v_x_co) + 'f' + '*v_x + ' +
          str(v_y_co) + 'f' + '*v_y + ' +
          str(v_z_co) + 'f' + '*v_z + ' +
         str(w_x_co) + 'f' + '*w_x + ' +
         str(w_y_co) + 'f' + '*w_y + ' +
         str(w_z_co) + 'f' + '*w_z' + ');')
    print()

o_z += dt*(-25.1644590643009f*o_z + 0.0f*theta + 0.0f*phi + 0.0f*n_x + 0.0f*n_y + 25.164459101855755f*r + 0.0f*g + 0.0f*a_z + 0.0f*v_x + 0.0f*v_y + 1.0f*v_z + 0.0f*w_x + 0.0f*w_y + 0.0f*w_z);

theta += dt*(0.0f*o_z + 0.0f*theta + 0.0f*phi + 0.016499999999950887f*n_x + 0.0f*n_y + 0.0f*r + 0.0f*g + 0.0f*a_z + -0.13505433741988782f*v_x + 0.0f*v_y + 0.0f*v_z + 0.0f*w_x + 1.0675271687100576f*w_y + 0.0f*w_z);

phi += dt*(0.0f*o_z + 0.0f*theta + 0.0f*phi + 0.0f*n_x + -0.020999999999958163f*n_y + 0.0f*r + 0.0f*g + 0.0f*a_z + 0.0f*v_x + 0.17188733854004568f*v_y + 0.0f*v_z + 1.0859436692699092f*w_x + 0.0f*w_y + 0.0f*w_z);

v_x += dt*(0.0f*o_z + 9.80999999999949f*theta + 0.0f*phi + 0.25337734850018023f*n_x + 0.0f*n_y + 0.0f*r + 0.0f*g + 0.0f*a_z + -2.0739218133198847f*v_x + 0.0f*v_y + 0.0f*v_z + 0.0f*w_x + 1.0369609066599423f*w_y + 0.0f*w_z);

v_y += dt*(0.0f*o_z + 0.0f*theta + -10.0f*phi + 0.0f*n_x + 0.25420068660014294f*n_y + 0.0f*r + 0.0f*g + 0.0f*a_z + 0.0f*v_x + -2.080660927359986f*v_y + 0.0